In [ ]:
import struct
import numpy as np
import cv2

def read_pod(file, fmt):
    # Read and unpack binary data according to the given format
    return struct.unpack(fmt, file.read(struct.calcsize(fmt)))

def read_pod_vector(file, fmt):
    # Read the size of the vector
    size, = read_pod(file, 'l')
    # Read each item in the vector
    return [read_pod(file, fmt) for _ in range(size)]

def read_keypoints(file_name):
    with open(file_name, 'rb') as file:
        # Assuming keypoints are stored as (x, y, size, angle, response, octave, class_id)
        num_keypoints, = read_pod(file, 'l')  # Read the number of keypoints
        keypoints = []
        for _ in range(num_keypoints):
            x, y, size, angle, response, octave, class_id = read_pod(file, 'fffffii')
            keypoints.append(cv2.KeyPoint(x, y, size, angle, response, octave, class_id))
        return keypoints

def read_kpt_matches(file_name):
    with open(file_name, 'rb') as file:
        # Assuming matches are stored as (queryIdx, trainIdx, imgIdx, distance)
        num_matches, = read_pod(file, 'l')  # Read the number of matches
        matches = []
        for _ in range(num_matches):
            queryIdx, trainIdx, imgIdx, distance = read_pod(file, 'iiif')
            matches.append(cv2.DMatch(queryIdx, trainIdx, imgIdx, distance))
        return matches

# Example usage:
keypoints_source = read_keypoints('srcA.dat')
keypoints_ref =  read_keypoints('refA.dat')


matches = read_kpt_matches('matchA.dat')


print (keypoints_source[22].angle)


245.80465698242188


In [ ]:

def compute_ttc_camera(kpts_prev, kpts_curr, kpt_matches, frame_rate):
    # Compute distance ratios between all matched keypoints
    dist_ratios = []  # Stores the distance ratios for all keypoints between curr. and prev. frame
    for match1 in kpt_matches:
        kp_outer_curr = kpts_curr[match1.trainIdx]
        kp_outer_prev = kpts_prev[match1.queryIdx]

        for match2 in kpt_matches:
            kp_inner_curr = kpts_curr[match2.trainIdx]
            kp_inner_prev = kpts_prev[match2.queryIdx]

            dist_curr = np.linalg.norm(np.array(kp_outer_curr.pt) - np.array(kp_inner_curr.pt))
            dist_prev = np.linalg.norm(np.array(kp_outer_prev.pt) - np.array(kp_inner_prev.pt))

            if dist_prev > np.finfo(float).eps and dist_curr >= 100.0:
                dist_ratio = dist_curr / dist_prev
                dist_ratios.append(dist_ratio)

    # Only continue if list of distance ratios is not empty
    if len(dist_ratios) == 0:
        return np.nan

    # Use medianDistRatio as more robust than meanDistRatio
    median_dist_ratio = np.median(dist_ratios)

    dT = 1 / frame_rate
    ttc = -dT / (1 - median_dist_ratio)

    return ttc


In [ ]:
ttc = compute_ttc_camera(keypoints_source,keypoints_ref,matches,10.0)



In [ ]:
print(ttc)

12.454468169322238
